In [ ]:
# ==================================================
# 2일차 - 1교시: 데이터 더 깊이 이해하기 & 그룹별 분석
# ==================================================
print("--- 2일차 1교시 시작 ---")

# 1. 필요한 라이브러리 불러오기 (Pandas, NumPy는 이전 차시에서 임포트 가정)
# 만약 Colab 세션을 새로 시작했다면 아래 두 줄의 주석을 해제하고 실행하세요.
# import pandas as pd
# import numpy as np

# [텍스트 셀 안내]
# (df_auto는 1일차에서 생성 및 'price' 컬럼 전처리가 완료되었다고 가정합니다.)
# 만약 df_auto가 없다면, 1일차의 해당 코드들을 먼저 실행해주세요!

# ---!!! 중요: 이번 시간 실습에 필요한 주요 컬럼 전처리 !!!---
# 'horsepower', 'num-of-doors', 'city-mpg', 'highway-mpg', 'engine-size', 'curb-weight' 컬럼이
# 이번 시간 실습에서 자주 사용됩니다. 이 컬럼들에 대한 전처리를 먼저 수행합니다.

# 전처리할 숫자형 예상 컬럼 리스트
numeric_cols_to_clean = ['horsepower', 'engine-size', 'curb-weight', 'city-mpg', 'highway-mpg', 'bore', 'stroke', 'peak-rpm', 'normalized-losses']

print("\n--- 주요 숫자 컬럼 전처리 시작 ---")
for col in numeric_cols_to_clean:
    if col in df_auto.columns:
        # '?'를 NaN으로 (이미 read_csv에서 na_values로 처리했을 수 있지만, 안전하게 한 번 더)
        if df_auto[col].dtype == 'object': # object 타입일 때만 replace 시도
            df_auto.loc[:, col] = df_auto[col].replace('?', np.nan)
        
        # 숫자형으로 변환 (오류 시 NaN)
        df_auto.loc[:, col] = pd.to_numeric(df_auto[col], errors='coerce')
        
        # float으로 명시적 변환 (이미 숫자형(int64)일 수 있으므로, float이 아닌 경우만 시도)
        # pd.to_numeric은 이미 float64 또는 int64로 변환하므로, 이 조건은 좀 더 엄밀하게
        if not pd.api.types.is_numeric_dtype(df_auto[col]) or df_auto[col].dtype == 'object':
             # 이 부분은 to_numeric 이후에는 거의 실행되지 않겠지만, 혹시 모를 경우를 대비
            try:
                df_auto.loc[:, col] = df_auto[col].astype(float)
            except ValueError:
                print(f"경고: '{col}'을 float으로 변환 중 오류. 추가 클리닝 필요.")

        # NaN을 평균으로 채우기 (숫자형으로 변환된 후 수행)
        if df_auto[col].isnull().sum() > 0: # NaN이 있을 경우에만 실행
            avg_val = df_auto[col].mean()
            if pd.isna(avg_val): # 평균 계산이 불가능한 경우 (모든 값이 NaN 등)
                df_auto.loc[:, col] = df_auto[col].fillna(0) # 평균 계산 안되면 0으로
                print(f"'{col}' 컬럼: NaN을 0으로 채움 (평균 계산 불가). 최종 Dtype: {df_auto[col].dtype}")
            else:
                df_auto.loc[:, col] = df_auto[col].fillna(avg_val)
                print(f"'{col}' 컬럼: NaN을 평균({avg_val:.2f})으로 채움. 최종 Dtype: {df_auto[col].dtype}")
    else:
        print(f"경고: '{col}' 컬럼이 df_auto에 없습니다.")
print("--- 주요 숫자 컬럼 전처리 완료 ---")


# 'num-of-doors' 범주형 컬럼 전처리 (결측치 최빈값으로)
if 'num-of-doors' in df_auto.columns:
    print("\n--- 'num-of-doors' 컬럼 전처리 시작 ---")
    # '?'를 NaN으로 (이미 read_csv에서 na_values로 처리했을 수 있지만, 안전하게 한 번 더)
    if df_auto['num-of-doors'].dtype == 'object':
        df_auto.loc[:, 'num-of-doors'] = df_auto['num-of-doors'].replace('?', np.nan)

    if df_auto['num-of-doors'].isnull().sum() > 0:
        most_frequent_doors = df_auto['num-of-doors'].mode()[0] # 최빈값
        df_auto.loc[:, 'num-of-doors'] = df_auto['num-of-doors'].fillna(most_frequent_doors)
        print(f"'num-of-doors' 컬럼: NaN을 최빈값('{most_frequent_doors}')으로 채움.")
    print("--- 'num-of-doors' 컬럼 전처리 완료 ---")
else:
    print("경고: 'num-of-doors' 컬럼이 df_auto에 없습니다.")


# 전처리 후 데이터 상태 확인
print("\n--- 모든 주요 컬럼 전처리 후 df_auto 종합 정보 (info) ---")
df_auto.info()


# --- 데이터 선택 및 필터링 실습 ---
print("\n\n--- 데이터 선택 및 필터링 실습 ---")

# 2. 특정 열 선택하기
if 'make' in df_auto.columns:
    make_col_d2 = df_auto['make']
    print("\n--- 'make' 컬럼 (앞 3개) ---")
    print(make_col_d2.head(3))

if all(col in df_auto.columns for col in ['make', 'price']):
    make_price_cols_d2 = df_auto[['make', 'price']]
    print("\n--- 'make', 'price' 컬럼 (앞 3개) ---")
    print(make_price_cols_d2.head(3))

# 3. 조건에 맞는 행 선택하기 (필터링)
if 'drive-wheels' in df_auto.columns:
    rwd_cars_df_d2 = df_auto[df_auto['drive-wheels'] == 'rwd']
    print("\n--- 'drive-wheels'가 'rwd'인 자동차 (앞 3개) ---")
    if all(col in rwd_cars_df_d2.columns for col in ['make', 'drive-wheels', 'price']):
        print(rwd_cars_df_d2[['make', 'drive-wheels', 'price']].head(3))
    print(f"'rwd' 자동차 총 대수: {len(rwd_cars_df_d2)}")


# --- 데이터 그룹화 (groupby) 실습 ---
print("\n\n--- 데이터 그룹화 (groupby) 실습 ---")

# 4. 'body-style'별 평균 'horsepower' 계산
if 'body-style' in df_auto.columns and 'horsepower' in df_auto.columns and pd.api.types.is_numeric_dtype(df_auto['horsepower']):
    avg_hp_by_style_d2 = df_auto.groupby('body-style')['horsepower'].mean().sort_values(ascending=False)
    print("\n--- 차체 스타일별 평균 마력 ---")
    print(avg_hp_by_style_d2)
else:
    print("\n'horsepower' 또는 'body-style' 컬럼 문제로 그룹별 평균 마력 계산 불가.")

# 5. 'fuel-type'별 평균 'price' 계산
if 'fuel-type' in df_auto.columns and 'price' in df_auto.columns and pd.api.types.is_numeric_dtype(df_auto['price']):
    avg_price_by_fuel_d2 = df_auto.groupby('fuel-type')['price'].mean().sort_values(ascending=False)
    print("\n--- 연료 종류별 평균 가격 ---")
    print(avg_price_by_fuel_d2)
else:
    print("\n'price' 또는 'fuel-type' 컬럼 문제로 그룹별 평균 가격 계산 불가.")


# --- 피벗 테이블 (pivot_table) 실습 ---
print("\n\n--- 피벗 테이블 실습 ---")

# 6. 'drive-wheels'을 행으로, 'num-of-doors'를 열로, 평균 'horsepower'를 값으로 하는 피벗 테이블
# 'num-of-doors'는 범주형, 'horsepower'는 숫자형으로 전처리 되어야 함
if all(col in df_auto.columns for col in ['drive-wheels', 'num-of-doors', 'horsepower']) and pd.api.types.is_numeric_dtype(df_auto['horsepower']):
    pivot_avg_hp_d2 = pd.pivot_table(df_auto,
                                     values='horsepower',
                                     index='drive-wheels',
                                     columns='num-of-doors', # 'num-of-doors'는 범주형이어야 함
                                     aggfunc='mean',
                                     fill_value=0)
    print("\n--- 피벗 테이블: 구동방식 및 문 개수별 평균 마력 ---")
    print(pivot_avg_hp_d2)
else:
    print("\n피벗 테이블 생성에 필요한 컬럼('drive-wheels', 'num-of-doors', 'horsepower') 문제 또는 'horsepower' 타입 문제.")


# --- 상관 관계 (correlation) 실습 ---
print("\n\n--- 상관 관계 실습 ---")

# 7. 주요 숫자형 컬럼들 간의 상관계수 행렬 계산
# 이전에 정의한 numeric_features_corr 리스트 사용 또는 새로 정의
numeric_features_corr_d2 = ['wheel-base', 'length', 'width', 'height', 'curb-weight',
                            'engine-size', 'horsepower', 'city-mpg', 'highway-mpg', 'price']
# df_auto에 실제 존재하고, 숫자형인 컬럼만 선택
valid_numeric_features_d2 = [col for col in numeric_features_corr_d2 if col in df_auto.columns and pd.api.types.is_numeric_dtype(df_auto[col])]

if len(valid_numeric_features_d2) > 1:
    df_numeric_for_corr_d2 = df_auto[valid_numeric_features_d2].dropna() # NaN 있는 행 제거하고 상관계수 계산
    if not df_numeric_for_corr_d2.empty and len(df_numeric_for_corr_d2.columns) > 1:
        correlation_matrix_d2 = df_numeric_for_corr_d2.corr()
        print("\n--- 주요 숫자형 컬럼 간 상관계수 행렬 ---")
        print(correlation_matrix_d2)

        if 'price' in correlation_matrix_d2.columns:
            price_correlations_d2 = correlation_matrix_d2['price'].sort_values(ascending=False)
            print("\n--- 'price'와 다른 변수들 간의 상관계수 (정렬) ---")
            print(price_correlations_d2)
        else:
            print("\n'price' 컬럼이 최종 상관 행렬에 포함되지 않았습니다 (전처리 또는 NaN 문제 확인).")
    else:
        print("\n상관관계 계산을 위한 데이터가 부족합니다 (NaN 제거 후 확인).")
else:
    print("\n상관관계 계산에 필요한 숫자형 컬럼이 2개 미만입니다.")

print("\n--- 2일차 1교시 실습 종료 ---")

# ==================================================
# 2일차 - 2교시: 데이터로 그림 그리기 & 탐색적 데이터 분석 맛보기 (모든 경고 수정된 전체 코드)
# ==================================================
print("\n--- 2일차 2교시 시작 ---")

# 1. 필요한 라이브러리 (Pandas, NumPy는 이전 차시에서, Matplotlib, Seaborn은 여기서)
# 만약 Colab 세션을 새로 시작했다면 아래 주석을 해제하고 Pandas, NumPy를 먼저 임포트하세요.
# import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline # This is a magic command, typically run in a separate cell in Jupyter

# [텍스트 셀 안내]
# (df_auto는 이전 교시에서 주요 컬럼 전처리가 완료되었다고 가정합니다.)
# 만약 df_auto 또는 필요한 컬럼의 전처리가 미흡하다면, 이전 교시 코드를 먼저 실행/수정해주세요!

# ---!!! 중요: 이번 시간 시각화에 필요한 추가 컬럼 전처리 !!!---
# (이전 답변의 전처리 코드와 동일하게 유지)
cols_for_viz_d2_warn_fixed = ['curb-weight', 'peak-rpm', 'highway-mpg', 'city-mpg']
print("\n--- 추가 시각화용 숫자 컬럼 전처리 시작 ---")
for col in cols_for_viz_d2_warn_fixed:
    if col in df_auto.columns:
        if df_auto[col].dtype == 'object':
            df_auto.loc[:, col] = df_auto[col].replace('?', np.nan)
            df_auto.loc[:, col] = pd.to_numeric(df_auto[col], errors='coerce') # Applied after replace, if object
            
            # This block executes only if the column was initially 'object',
            # then converted by to_numeric, and then its dtype is checked.
            if df_auto[col].dtype != 'float64' and df_auto[col].dtype != 'int64':
                try:
                    df_auto.loc[:, col] = df_auto[col].astype(float)
                except ValueError:
                    print(f"경고: '{col}' 컬럼을 float으로 변환 중 오류 발생. 추가 클리닝이 필요할 수 있습니다.")
                    def force_float_or_nan_viz_wf(x): # 함수 이름 변경 (중복 방지)
                        try: return float(x)
                        except: return np.nan
                    df_auto.loc[:, col] = df_auto[col].apply(force_float_or_nan_viz_wf).astype(float)
                    
                    # NaN filling logic, as per original deep indentation,
                    # seems to apply only within this specific except block.
                    avg_val = df_auto[col].mean()
                    if pd.isna(avg_val):
                        df_auto.loc[:, col] = df_auto[col].fillna(0)
                        print(f"'{col}' 컬럼: NaN을 0으로 채움 (평균 계산 불가). 최종 Dtype: {df_auto[col].dtype}")
                    else:
                        df_auto.loc[:, col] = df_auto[col].fillna(avg_val)
                        print(f"'{col}' 컬럼: NaN을 평균({avg_val:.2f})으로 채움. 최종 Dtype: {df_auto[col].dtype}")
        # else: # If not object type initially, this specific processing chain is skipped.
        #     # If there was NaN filling or other logic intended for non-object numeric columns,
        #     # it would need to be structured differently.
        #     # For simplicity of fixing indentation, I'm keeping the original implied structure.
        #     pass # No explicit else action in the original structure for non-object types at this point.

    else: # This else is for `if col in df_auto.columns:`
        print(f"경고: '{col}' 컬럼이 df_auto에 없습니다.")
print("--- 추가 시각화용 숫자 컬럼 전처리 완료 ---")


# 2. Matplotlib으로 기본 그래프 그리기 실습
print("\n--- Matplotlib 산점도: 'curb-weight' vs 'city-mpg' (색상/크기로 'price' 표현) ---")
if all(col in df_auto.columns and pd.api.types.is_numeric_dtype(df_auto[col]) for col in ['price', 'curb-weight', 'city-mpg']):
    df_scatter_viz_wf = df_auto[['curb-weight', 'city-mpg', 'price']].dropna()
    if not df_scatter_viz_wf.empty and (df_scatter_viz_wf['price'] > 0).any():
        s_values_viz_wf = (df_scatter_viz_wf['price'].clip(lower=1)) / 150

        plt.figure(figsize=(12, 7))
        scatter_plot_fancy_d2_viz_wf = plt.scatter(df_scatter_viz_wf['curb-weight'], df_scatter_viz_wf['city-mpg'],
                                                   c=df_scatter_viz_wf['price'],
                                                   s=s_values_viz_wf,
                                                   cmap='plasma', alpha=0.6, edgecolors='w', linewidth=0.5)
        plt.title('Curb Weight vs City MPG (Color & Size by Price)', fontsize=15)
        plt.xlabel('Curb Weight (lbs)', fontsize=12)
        plt.ylabel('City MPG', fontsize=12)
        plt.grid(True, linestyle=':', alpha=0.5)
        try:
            cbar = plt.colorbar(scatter_plot_fancy_d2_viz_wf, label='Price (USD)')
        except Exception as e:
            print(f"컬러바 생성 중 오류: {e}")
        plt.show()
    else:
        print("산점도(색상/크기 포함)를 위한 유효한 'price' 데이터가 부족합니다.")
else:
    print("산점도 그리기에 필요한 컬럼('price', 'curb-weight', 'city-mpg') 중 일부가 없거나 숫자형이 아닙니다.")


# 3. Seaborn으로 더 멋진 통계 그래프 그리기 실습
print("\n--- Seaborn 박스 플롯: 'body-style'별 'highway-mpg' 분포 (경고 수정) ---")
if 'body-style' in df_auto.columns and 'highway-mpg' in df_auto.columns and pd.api.types.is_numeric_dtype(df_auto['highway-mpg']):
    plt.figure(figsize=(12, 7))
    sns.boxplot(x='body-style',
                y='highway-mpg',
                hue='body-style', # x 변수를 hue에도 할당
                data=df_auto.dropna(subset=['highway-mpg', 'body-style']), # hue 사용 시 hue 컬럼도 dropna 대상
                palette='pastel',
                legend=False)     # 범례는 표시하지 않음 (x축 레이블로 충분)
    plt.title('Highway MPG Distribution by Body Style', fontsize=15)
    plt.xlabel('Body Style', fontsize=12)
    plt.ylabel('Highway MPG', fontsize=12)
    plt.xticks(rotation=30)
    plt.grid(True, linestyle=':', alpha=0.5)
    plt.show()
else:
    print("박스 플롯 그리기에 필요한 컬럼('body-style', 'highway-mpg') 중 일부가 없거나 'highway-mpg'가 숫자형이 아닙니다.")


print("\n--- Seaborn 회귀 플롯: 'horsepower' vs 'peak-rpm' 관계 ---")
# 'horsepower'와 'peak-rpm' 컬럼은 이전 또는 이 셀 상단에서 전처리됨
if 'horsepower' in df_auto.columns and 'peak-rpm' in df_auto.columns:
    if pd.api.types.is_numeric_dtype(df_auto['horsepower']) and pd.api.types.is_numeric_dtype(df_auto['peak-rpm']):
        plt.figure(figsize=(10, 6))
        df_reg_subset_wf = df_auto[['horsepower', 'peak-rpm']].dropna()
        if not df_reg_subset_wf.empty:
            sns.regplot(x='horsepower', y='peak-rpm', data=df_reg_subset_wf,
                        scatter_kws={'color':'teal',
                                     'alpha':0.4,
                                     's':40,
                                     'edgecolor':'black',
                                     'linewidths':0.5},
                        line_kws={'color':'darkred',
                                  'linewidth':2,
                                  'linestyle':'--'})
            plt.title('Horsepower vs Peak RPM with Regression Line', fontsize=15)
            plt.xlabel('Horsepower', fontsize=12)
            plt.ylabel('Peak RPM', fontsize=12)
            plt.grid(True, linestyle=':', alpha=0.5)
            plt.show()
        else:
            print("회귀 플롯을 위한 유효 데이터가 없습니다 ('horsepower' 또는 'peak-rpm'에 NaN이 많을 수 있음).")
    else:
        print("회귀 플롯을 위한 'horsepower' 또는 'peak-rpm' 컬럼이 숫자형이 아닙니다. 전처리를 확인하세요.")
else:
    print("회귀 플롯 그리기에 필요한 컬럼('horsepower', 'peak-rpm') 중 일부가 df_auto에 없습니다.")


# 4. (추가 실습) Seaborn의 pairplot
print("\n--- Seaborn Pair Plot (주요 숫자 컬럼 간 관계) (경고 수정) ---")
cols_for_pairplot_d2_full_wf = ['price', 'horsepower', 'engine-size', 'highway-mpg', 'curb-weight']
valid_cols_pairplot_wf = [col for col in cols_for_pairplot_d2_full_wf if col in df_auto.columns and pd.api.types.is_numeric_dtype(df_auto[col])]

if len(valid_cols_pairplot_wf) > 1:
    df_pairplot_subset_d2_wf = df_auto[valid_cols_pairplot_wf].dropna()
    if not df_pairplot_subset_d2_wf.empty and len(df_pairplot_subset_d2_wf.columns) > 1:
        print(f"Pairplot 대상 컬럼: {df_pairplot_subset_d2_wf.columns.tolist()}")
        print(f"Pairplot 데이터 shape (NaN 제거 후): {df_pairplot_subset_d2_wf.shape}")
        if len(df_pairplot_subset_d2_wf) >= 2 :
            sns.pairplot(df_pairplot_subset_d2_wf, diag_kind='kde',
                         plot_kws={'alpha':0.5, 's':20, 'edgecolor':'k'},
                         diag_kws={'linewidth':1.5, 'fill':True}) # 'shade'를 'fill'로 변경
            plt.suptitle('Pair Plot of Key Car Features', y=1.02, fontsize=16)
            plt.show()
        else:
            print("Pairplot을 그리기에 데이터가 너무 부족합니다 (NaN 제거 후 2행 미만).")
    else:
        print("Pairplot을 위한 유효 데이터가 없습니다 (NaN 또는 컬럼 타입 문제).")
else:
    print("Pairplot에 필요한 숫자형 컬럼이 2개 미만입니다. 전처리 상태를 확인해주세요.")

print("\n--- 2일차 2교시 실습 종료 ---")

# ==================================================
# 2일차 - 3교시: 모델링 첫걸음 & 데이터 과학 여정 마무리 (UserWarning 수정된 전체 코드)
# ==================================================
print("\n--- 2일차 3교시 시작 ---")

# 1. 필요한 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split # 3일차에 사용
# from sklearn.metrics import mean_squared_error, r2_score # 3일차에 사용
# %matplotlib inline # Magic command, typically run in its own cell in Jupyter

# [텍스트 셀 안내]
# (df_auto는 이전 교시에서 주요 컬럼 전처리가 완료되었다고 가정합니다.)
# 'engine-size'와 'price' 컬럼이 숫자형이고 NaN이 처리된 상태여야 합니다.

# ---!!! 중요: 모델링에 사용할 'engine-size'와 'price' 컬럼 최종 확인 및 전처리 !!!---
# (이전 교시의 전처리 코드가 이미 실행되었다고 가정, 필요시 이전 셀 실행 또는 아래 코드 실행)
print("\n--- 모델링용 'engine-size', 'price' 컬럼 최종 전처리 시작 (필요시) ---")
cols_for_model_d2_warnfix = ['engine-size', 'price']
for col in cols_for_model_d2_warnfix:
    if col in df_auto.columns:
        # '?' 처리 (na_values로 이미 처리되었을 수 있음)
        # This specific structure for processing is from the original indentation
        if df_auto[col].dtype == 'object' and df_auto[col].astype(str).str.contains('\?').any():
            df_auto.loc[:, col] = df_auto[col].replace('?', np.nan)
            # 숫자형 변환 (indented under the 'object' and '?' condition)
            df_auto.loc[:, col] = pd.to_numeric(df_auto[col], errors='coerce')
            # float 타입으로 명시적 변환 (indented further)
            if df_auto[col].dtype != 'float64' and df_auto[col].dtype != 'int64':
                try:
                    df_auto.loc[:, col] = df_auto[col].astype(float)
                except ValueError:
                    print(f"경고: '{col}' 컬럼 float 변환 오류. 추가 클리닝 필요.")
                    def force_float_or_nan_mod(x): # 함수 이름 변경
                        try: return float(x)
                        except: return np.nan
                    df_auto.loc[:, col] = df_auto[col].apply(force_float_or_nan_mod).astype(float)
                    # NaN을 평균으로 채우기 (indented under except, this is very specific)
                    avg_val = df_auto[col].mean()
                    if pd.isna(avg_val):
                        df_auto.loc[:, col] = df_auto[col].fillna(0)
                    else:
                        df_auto.loc[:, col] = df_auto[col].fillna(avg_val)
                print(f"'{col}' 컬럼 전처리 완료. Dtype: {df_auto[col].dtype}")
        # Note: If a column is numeric with NaNs but wasn't object or didn't have '?',
        # the above fillna logic is skipped. This might be unintentional if general NaN fill was needed.
        # However, the problem asks to fix indentation, not to rewrite fundamental logic.
        # The main `if` for modeling later will check for numeric types.
    else:
        print(f"경고: 모델링에 필요한 '{col}' 컬럼이 df_auto에 없습니다.")
print("--- 모델링용 컬럼 최종 전처리 완료 ---")

# info()로 최종 상태 확인
print("\n--- 최종 전처리 후 df_auto.info() ---")
df_auto.info()


# 2. 입력(X)과 출력(Y) 데이터 준비하기 ('engine-size'로 'price' 예측)
# Check if necessary columns exist and are numeric before proceeding with modeling
if 'engine-size' in df_auto.columns and 'price' in df_auto.columns and \
   pd.api.types.is_numeric_dtype(df_auto['engine-size']) and \
   pd.api.types.is_numeric_dtype(df_auto['price']):

    print("\n--- 입력(X)과 출력(Y) 데이터 준비 ---")
    # 입력 X는 반드시 2차원 배열 (Pandas 데이터프레임) 형태! 컬럼 이름을 가집니다.
    X_engine_d2_wf = df_auto[['engine-size']]
    # 출력 Y는 1차원 배열 (Pandas 시리즈) 형태
    Y_price_d2_wf = df_auto['price']

    print("Shape of X_engine_d2_wf:", X_engine_d2_wf.shape)
    print("Shape of Y_price_d2_wf:", Y_price_d2_wf.shape)
    print("Feature names in X_engine_d2_wf:", X_engine_d2_wf.columns.tolist())


    # 3. 단순 선형 회귀 모델 만들고 학습시키기
    print("\n--- 단순 선형 회귀 모델 학습 ---")
    lm_engine_price_d2_wf = LinearRegression()    # 모델 객체 생성
    # .fit() 할 때 X 데이터프레임은 컬럼 이름을 가집니다.
    lm_engine_price_d2_wf.fit(X_engine_d2_wf, Y_price_d2_wf) # 모델 학습
    print("모델 학습 완료!")
    if hasattr(lm_engine_price_d2_wf, 'feature_names_in_'):
        print(f"모델이 학습한 특성 이름: {lm_engine_price_d2_wf.feature_names_in_}")


    # 4. 학습된 모델로 예측하기
    # .predict() 할 때도 동일한 컬럼 이름을 가진 데이터프레임을 사용하는 것이 좋습니다.
    Y_pred_engine_price_d2_wf = lm_engine_price_d2_wf.predict(X_engine_d2_wf)
    print("\n--- 학습된 모델로 가격 예측 (앞 5개) ---")
    for i in range(min(5, len(X_engine_d2_wf))):
        print(f"엔진 크기: {X_engine_d2_wf.iloc[i].values[0]}, 실제 가격: ${Y_price_d2_wf.iloc[i]:.2f}, 예측 가격: ${Y_pred_engine_price_d2_wf[i]:.2f}")


    # 5. 학습된 모델의 기울기와 절편 확인하기
    slope_d2_wf = lm_engine_price_d2_wf.coef_[0]
    intercept_d2_wf = lm_engine_price_d2_wf.intercept_
    print("\n--- 학습된 모델의 파라미터 ---")
    print(f"기울기 (engine-size 계수): {slope_d2_wf:.2f}")
    print(f"y절편: {intercept_d2_wf:.2f}")
    print(f"예측 모델: Price ≈ {slope_d2_wf:.2f} * EngineSize + {intercept_d2_wf:.2f}")


    # 6. 모델 평가: 예측값과 실제값 시각적으로 비교하기
    print("\n--- 모델 예측 결과 시각화 (Seaborn regplot) ---")
    plt.figure(figsize=(10, 6))
    # regplot은 내부적으로 데이터프레임에서 컬럼명으로 x, y를 찾으므로 안전합니다.
    df_plot_reg = df_auto[['engine-size', 'price']].dropna() # 시각화용 데이터 NaN 제거
    if not df_plot_reg.empty:
        sns.regplot(x='engine-size', y='price', data=df_plot_reg,
                    scatter_kws={'alpha':0.4, 'color':'mediumpurple', 's':50, 'edgecolor':'k'},
                    line_kws={'color':'crimson', 'linewidth':2.5, 'label':f'y={slope_d2_wf:.2f}x + {intercept_d2_wf:.2f}'})
        plt.title('Engine Size vs Price (Actual Data and Regression Line)', fontsize=15)
        plt.xlabel('Engine Size', fontsize=12)
        plt.ylabel('Price (USD)', fontsize=12)
        plt.legend()
        plt.grid(True, linestyle=':', alpha=0.6)
        plt.show()
    else:
        print("시각화를 위한 유효 데이터가 없습니다.")


    # (참고) 모델 성능 수치로 확인하기 (R-제곱 값)
    # .score() 할 때도 X 데이터프레임은 컬럼 이름을 가져야 합니다.
    r_squared_d2_wf = lm_engine_price_d2_wf.score(X_engine_d2_wf, Y_price_d2_wf)
    print(f"\n모델의 R-squared (결정 계수): {r_squared_d2_wf:.4f}")


    # 7. 새로운 '엔진 크기'로 '가격' 예측해 보기 (UserWarning 해결을 위한 수정)
    print("\n--- 새로운 엔진 크기로 가격 예측 (UserWarning 수정) ---")
    new_engine_values_list = [[100], [150], [200], [250]] # 예측하고 싶은 새로운 엔진 크기 리스트

    # NumPy 배열 대신 Pandas 데이터프레임으로 만들고, 학습 시 사용한 컬럼 이름을 명시적으로 부여합니다.
    new_engine_df_wf = pd.DataFrame(new_engine_values_list, columns=X_engine_d2_wf.columns)
    # 또는 columns=['engine-size'] 로 직접 지정 가능
    print("새로운 예측용 X 데이터 (데이터프레임 형태):")
    print(new_engine_df_wf)
    print("새로운 예측용 X 데이터의 특성 이름:", new_engine_df_wf.columns.tolist())

    predicted_new_prices_wf = lm_engine_price_d2_wf.predict(new_engine_df_wf)

    for i in range(len(new_engine_df_wf)):
        print(f"엔진 크기가 {new_engine_df_wf.iloc[i, 0]}일 때, 예상 가격은: ${predicted_new_prices_wf[i]:.2f} 입니다.")

else: # This 'else' corresponds to the main 'if' checking for 'engine-size' and 'price' columns
    print("모델링에 필요한 'engine-size' 또는 'price' 컬럼이 없거나 숫자형이 아닙니다. 이전 단계 전처리를 확인해주세요.")


print("\n--- 2일차 3교시 실습 종료 ---")